In [1]:
import pandas as pd
import numpy as np

#loading datasets
apps = pd.read_csv("googleplaystore.csv")
reviews = pd.read_csv("googleplaystore_user_reviews.csv")

print("Apps shape:", apps.shape)
print("Reviews shape:", reviews.shape)


Apps shape: (10841, 13)
Reviews shape: (64295, 5)


In [2]:
apps['Price'] = apps['Price'].astype(str).str.strip()

print("Unique non-numeric Price samples:")
print(apps[~apps['Price'].str.replace(r'[\d\.\$]', '', regex=True).str.strip().eq('')]['Price'].value_counts().head(20))


Unique non-numeric Price samples:
Everyone    1
Name: Price, dtype: int64


In [3]:
apps.head(295)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,TurboScan: scan documents and receipts in PDF,BUSINESS,4.7,11442,6.8M,"100,000+",Paid,$4.99,Everyone,Business,"March 25, 2018",1.5.2,4.0 and up
291,Tiny Scanner Pro: PDF Doc Scan,BUSINESS,4.8,10295,39M,"100,000+",Paid,$4.99,Everyone,Business,"April 11, 2017",3.4.6,3.0 and up
292,Docs To Go™ Free Office Suite,BUSINESS,4.1,217730,Varies with device,"50,000,000+",Free,0,Everyone,Business,"April 2, 2018",Varies with device,Varies with device
293,OfficeSuite : Free Office + PDF Editor,BUSINESS,4.3,1002859,35M,"100,000,000+",Free,0,Everyone,Business,"August 2, 2018",9.7.14188,4.1 and up


In [4]:
apps['Price'].unique()

array(['0', '$4.99', '$3.99', '$6.99', '$1.49', '$2.99', '$7.99', '$5.99',
       '$3.49', '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49',
       '$10.00', '$24.99', '$11.99', '$79.99', '$16.99', '$14.99',
       '$1.00', '$29.99', '$12.99', '$2.49', '$10.99', '$1.50', '$19.99',
       '$15.99', '$33.99', '$74.99', '$39.99', '$3.95', '$4.49', '$1.70',
       '$8.99', '$2.00', '$3.88', '$25.99', '$399.99', '$17.99',
       '$400.00', '$3.02', '$1.76', '$4.84', '$4.77', '$1.61', '$2.50',
       '$1.59', '$6.49', '$1.29', '$5.00', '$13.99', '$299.99', '$379.99',
       '$37.99', '$18.99', '$389.99', '$19.90', '$8.49', '$1.75',
       '$14.00', '$4.85', '$46.99', '$109.99', '$154.99', '$3.08',
       '$2.59', '$4.80', '$1.96', '$19.40', '$3.90', '$4.59', '$15.46',
       '$3.04', '$4.29', '$2.60', '$3.28', '$4.60', '$28.99', '$2.95',
       '$2.90', '$1.97', '$200.00', '$89.99', '$2.56', '$30.99', '$3.61',
       '$394.99', '$1.26', 'Everyone', '$1.20', '$1.04'], dtype=object)

In [5]:
                   ###Cleaning googleplaystore data

#Drop Duplicates
apps.drop_duplicates(inplace=True)

#Drop rows with missing Rating 
apps.dropna(subset=['Rating'], inplace=True)

#Clean Size
# Convert M and k properly to MB
def size_to_mb(value):
    try:
        value = str(value).strip()
        if 'M' in value:
            return float(value.replace('M', ''))
        elif 'k' in value:
            return float(value.replace('k', '')) / 1024  # convert KB to MB
        elif value == 'Varies with device' or value == '':
            return np.nan
        else:
            return float(value)
    except:
        return np.nan
apps['Size'] = apps['Size'].apply(size_to_mb)
# Fill missing sizes with median
median_size = apps['Size'].median()
apps['Size'].fillna(median_size, inplace=True)

#Clean 'Installs'
apps['Installs'] = (
    apps['Installs']
    .astype(str)
    .str.replace('+', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.strip()
)
# Replace known invalid strings with NaN
apps['Installs'] = apps['Installs'].replace(['Everyone', 'Free', 'Paid', 'NaN', 'Varies with device'], np.nan)
# Convert to numeric safely
apps['Installs'] = pd.to_numeric(apps['Installs'], errors='coerce')

    
#Clean type
apps = apps[(apps['Type'].notna()) & (apps['Type'] != '0')].reset_index(drop=True)

#Clean Price
# Replace the single text anomaly
apps['Price'] = apps['Price'].replace('Everyone', np.nan)

# Remove dollar signs and convert to numeric
apps['Price'] = (
    apps['Price']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.strip()
)
# Convert to float, forcing invalid values to NaN
apps['Price'] = pd.to_numeric(apps['Price'], errors='coerce')

    
#Clean Content rating
apps['Content Rating'] = apps['Content Rating'].replace('Unrated', np.nan)
apps = apps.dropna(subset=['Content Rating'])
apps.reset_index(drop=True, inplace=True)

#convert 'last updated' to datetime format
apps['Last Updated'] = pd.to_datetime(apps['Last Updated'], errors='coerce')
#fill the NaN rows
median_date = apps['Last Updated'].median()
apps['Last Updated'].fillna(median_date, inplace=True)

import re
# Clean up 'Current Ver'
apps['Current Ver'] = (
    apps['Current Ver']
    .astype(str)
    .str.extract(r'(\d+\.\d+)')[0]
    .astype(float)
    .round(1)
)
# Fill NaN with median
median_current = apps['Current Ver'].median()
apps['Current Ver'].fillna(median_current, inplace=True)
print("Filled missing Current Ver with median:", median_current)

#Clean Android version
apps['Android Ver'] = (
    apps['Android Ver']
    .astype(str)
    .str.extract(r'(\d+\.\d+)')[0]   # extract only major.minor
    .astype(float)
    .round(1)                        # keep one decimal place
)
# Fill NaN with median
median_android = apps['Android Ver'].median()
apps['Android Ver'].fillna(median_android, inplace=True)
print("Filled missing Android Ver with median:", median_android)



Filled missing Current Ver with median: 2.1
Filled missing Android Ver with median: 4.1


In [7]:
apps.head(991)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000.0,Free,0.0,Everyone,Art & Design,2018-01-07,1.0,4.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000.0,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0,4.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000.0,Free,0.0,Everyone,Art & Design,2018-08-01,1.2,4.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000.0,Free,0.0,Teen,Art & Design,2018-06-08,2.1,4.2
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000.0,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,Delivery yogi.,FOOD_AND_DRINK,4.4,90042,14.0,10000000.0,Free,0.0,Everyone,Food & Drink,2018-07-18,2.1,4.1
987,Delivery trough - delivery trough delivery trough,FOOD_AND_DRINK,4.3,58316,14.0,5000000.0,Free,0.0,Everyone,Food & Drink,2018-07-25,2.1,4.1
988,Dr. Oetker recipe ideas,FOOD_AND_DRINK,4.2,8509,17.0,1000000.0,Free,0.0,Everyone,Food & Drink,2018-07-12,3.5,4.0
989,SONIC Drive-In,FOOD_AND_DRINK,4.3,19314,43.0,1000000.0,Free,0.0,Everyone,Food & Drink,2018-07-31,3.3,4.4


In [8]:
                               ###Cleaning googleplaystore_user_reviews data
# Remove duplicates
reviews.drop_duplicates(inplace=True)

# Check missing values
reviews.isnull().sum()


App                         0
Translated_Review         987
Sentiment                 982
Sentiment_Polarity        982
Sentiment_Subjectivity    982
dtype: int64

In [9]:
#Drop NaN translated_review
reviews = reviews.dropna(subset=['Translated_Review']).reset_index(drop=True)
# Fill missing Sentiment, Polarity, and Subjectivity if needed
reviews['Sentiment'].fillna('Neutral', inplace=True)
reviews['Sentiment_Polarity'].fillna(0, inplace=True)
reviews['Sentiment_Subjectivity'].fillna(0, inplace=True)

#Clean text data
def clean_text(text):
    text = text.lower()                               # lowercase
    text = re.sub(r'[^a-z\s]', '', text)              # remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()          # remove extra spaces
    return text

reviews['Clean_Review'] = reviews['Translated_Review'].apply(clean_text)

#Validate sentiment values
reviews['Sentiment'] = reviews['Sentiment'].str.capitalize()
valid_sentiments = ['Positive', 'Negative', 'Neutral']
reviews = reviews[reviews['Sentiment'].isin(valid_sentiments)]


In [57]:
print("Missing values after cleaning:\n", reviews.isnull().sum())
print("\nUnique Sentiments:", reviews['Sentiment'].unique())
reviews.head()


Missing values after cleaning:
 App                       0
Translated_Review         0
Sentiment                 0
Sentiment_Polarity        0
Sentiment_Subjectivity    0
Clean_Review              0
dtype: int64

Unique Sentiments: ['Positive' 'Neutral' 'Negative']


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Clean_Review
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333,i like eat delicious food thats im cooking foo...
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462,this help eating healthy exercise regular basis
2,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000,works great especially going grocery store
3,10 Best Foods for You,Best idea us,Positive,1.00,0.300000,best idea us
4,10 Best Foods for You,Best way,Positive,1.00,0.300000,best way


In [59]:
reviews.to_csv("cleaned_reviews.csv", index=False, encoding="utf-8", quoting=1)

In [10]:
# Drop redundant helper columns 
apps.drop(columns=['Major_Version', 'Android Num', 'Android_Num'], inplace=True, errors='ignore')

# ---- Preview cleaned version columns ----
apps[['App', 'Current Ver', 'Android Ver']].head(10)

,App,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,1.0,4.0
1,Coloring book moana,2.0,4.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",1.2,4.0
3,Sketch - Draw & Paint,2.1,4.2
4,Pixel Draw - Number Art Coloring Book,1.1,4.4
5,Paper flowers instructions,2.1,2.3
6,Smoke Effect Photo Maker - Smoke Editor,1.1,4.0
7,Infinite Painter,6.1,4.2
8,Garden Coloring Book,2.9,3.0
9,Kids Paint Free - Drawing Fun,2.8,4.0


In [12]:
apps['App'].value_counts().head(20)

ROBLOX                                              9
8 Ball Pool                                         7
Helix Jump                                          6
Bubble Shooter                                      6
Zombie Catchers                                     6
Candy Crush Saga                                    5
Bowmasters                                          5
Temple Run 2                                        5
slither.io                                          5
Duolingo: Learn Languages Free                      5
Subway Surfers                                      5
Granny                                              5
Angry Birds Classic                                 5
Farm Heroes Saga                                    5
Zombie Tsunami                                      5
Google Photos                                       4
My Talking Angela                                   4
Sniper 3D Gun Shooter: Free Shooting Games - FPS    4
BeautyPlus - Easy Photo Edit

In [13]:
apps[apps['App'].str.lower() == 'applock']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2798,Applock,TOOLS,4.4,4934130,14.0,100000000.0,Free,0.0,Everyone,Tools,2018-06-11,2.1,4.1
4415,AppLock,TOOLS,4.4,4931562,14.0,100000000.0,Free,0.0,Everyone,Tools,2018-06-11,2.1,4.1


In [14]:
# Convert 'Last Updated' to datetime
apps['Last Updated'] = pd.to_datetime(apps['Last Updated'], errors='coerce')

# Sort by App name and date, newest first
apps = apps.sort_values(['App', 'Last Updated'], ascending=[True, False])

# Drop duplicates — keep only the newest version per App
apps = apps.drop_duplicates(subset='App', keep='first')

print(apps.shape)

(8195, 13)


In [20]:
# Show invalid app names
invalid_apps = apps[apps['App'].str.contains(r'[#?/\\]|^\W|Ã|download', case=False, na=False)]
invalid_apps
invalid_apps.isna().sum()
invalid_apps.describe(include='all')

C:\Users\Dell\AppData\Local\Temp\ipykernel_7316\1958252208.py:5: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  invalid_apps.describe(include='all')


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
count,197,197,197.000000,1.970000e+02,197.000000,1.970000e+02,197,197.000000,197,197,197,197.000000,197.000000
unique,197,29,NaN,NaN,NaN,NaN,2,NaN,4,39,143,NaN,NaN
top,#name?,TOOLS,NaN,NaN,NaN,NaN,Free,NaN,Everyone,Tools,2018-08-02 00:00:00,NaN,NaN
freq,1,46,NaN,NaN,NaN,NaN,182,NaN,168,46,11,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-02 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-08-08 00:00:00,NaN,NaN
mean,NaN,NaN,4.152284,1.603282e+05,13.602142,5.996137e+06,NaN,2.326650,NaN,NaN,NaN,12.905584,3.977157
std,NaN,NaN,0.486719,1.281161e+06,15.272431,3.760229e+07,NaN,28.529895,NaN,NaN,NaN,143.613333,0.816373
min,NaN,NaN,1.700000,2.000000e+00,0.024414,1.000000e+01,NaN,0.000000,NaN,NaN,NaN,0.100000,1.500000
25%,NaN,NaN,3.900000,1.420000e+02,3.900000,1.000000e+04,NaN,0.000000,NaN,NaN,NaN,1.200000,4.000000


In [21]:
# Drop rows with invalid or corrupted app names
apps = apps[~apps['App'].str.contains(r'(^#|\.r|/u/|^\s*$|\?|Ã)', na=False)]

apps.dropna(subset=['App', 'Category', 'Rating', 'Installs', 'Size'], inplace=True)


C:\Users\Dell\AppData\Local\Temp\ipykernel_7316\101849415.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  apps = apps[~apps['App'].str.contains(r'(^#|\.r|/u/|^\s*$|\?|Ã)', na=False)]


In [15]:
apps['App'] = apps['App'].str.strip().str.lower()
apps = apps.drop_duplicates(subset='App', keep='first')

In [24]:
apps.to_csv("cleaned_Apps_New.csv", index=False)